In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.svm import SVR

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from datetime import datetime, timedelta
import ast

In [2]:
# Dataset de entrenamiento
df_Nizanda = pd.read_csv('./Datasets/Nizanda_T1.csv')
#df_Nizanda
df_Prueba = pd.read_csv('./Datasets/Nizanda_22_Prueba.csv')
#df_Prueba

In [18]:
# Creacion del diccionario resultados_LR
resSvr = {'Modelo': [], 'Horizon': [], 'R2': [], 'RMSE': [], 'MAE': [], 'Comb': [], 'Params': []}

# Transformacion de diccionario a Dataframe
df_SVR_mimo_test = pd.DataFrame(resSvr)
df_SVR_mimo_test

,Modelo,Horizon,R2,RMSE,MAE,Comb,Params


In [4]:
def data_preprocessing(df, scaler):        
    # Escalando el dataset
    data_scaled = scaler.transform(df)

    values = data_scaled
    
    if len(df.columns) == 1:
        values = values[:, 0]
    
    return values, len(df.columns)

In [5]:
def get_labeled_window(data, window_size, horizon, n_cols):
    if n_cols == 1:
        return data[:, :window_size], data[:, -horizon:]
    else:
        #print(data.shape)
        return data[:, :window_size], data[:, -horizon:, 3]
    
def make_windows(data, input_size, horizon, n_cols, shift = 0):
    # Crea una matriz 2D con el la longuitud del input size + horizon
    window_step = np.expand_dims(np.arange(input_size + horizon + shift), axis = 0)
    window_indexes = window_step + np.expand_dims(np.arange(len(data) - (shift + input_size + horizon - 1)), axis = 0).T
    windowed_array = data[window_indexes]
    
    windows, labels = get_labeled_window(windowed_array, window_size = input_size, horizon = horizon, n_cols = n_cols)
    
    return windows, labels

In [7]:
def run_experiment(data, window, horizon, shift, scaler, cv, model):
    w, n_cols = data_preprocessing(data, scaler)
    x_, y_ = make_windows(w, input_size = window, horizon = horizon, shift = shift, n_cols = n_cols)

    if len(x_.shape) > 2:
        # Convertir a arreglo de 2 dimensiones
        rows, nx, ny = x_.shape
        x_ = x_.reshape((rows, nx*ny))
        
    cv_results = cross_validate(
        model,
        x_,
        y_,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error", "r2"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    r2 = cv_results["test_r2"]
    print(
        f"Coefficient of determination: {r2.mean():.3f} +/- {r2.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}\n"
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
    )
    return {'r2': r2.mean(), 'rmse': rmse.mean(), 'mae': mae.mean()}

In [8]:
def run_test(data, window, horizon, shift, scaler, model):
    w, n_cols = data_preprocessing(data, scaler)
    x_, y_ = make_windows(w, input_size= window, horizon = horizon, shift = shift,  n_cols = n_cols)
    
    if len(x_.shape) > 2:
        # Convertir a arreglo de 2 dimensiones
        rows, nx, ny = x_.shape
        x_ = x_.reshape((rows, nx*ny))
    
    ### CAMBIAR LA FORMA DE LAS PRUEBAS
    ### AL HACER LA PRUEBA NO ES NECESARIO HACER EL FIT, DEBIDO A QUE SE TIENE QUE USAR EL MODELO DE ENTRENAMIENTO
    #model.fit(x_, y_)
    y_pred = model.predict(x_)
    
    r2 = r2_score(y_, y_pred)
    mae = mean_absolute_error(y_, y_pred)
    rmse = mean_squared_error(y_, y_pred, squared= False)
    return {'r2': r2, 'rmse': rmse, 'mae': mae}, y_pred

In [9]:
# Graficando la regresion lineal dentro de la serie de tiempo del viento
def plot_series(Y_true, Y_pred, title = 'Serie de Tiempo de la velocidad del viento', vline = False):
    gris_uat = '#CFCDC9' #(207, 205, 201)
    azul_uat = '#4070B6' #(0, 66, 106)
    naranja_uat = '#ECB06F' #(184, 97, 106)
    #gris_uat_a = '#CFCDC9' #(207, 205, 201)
    
    
    fig, ax = plt.subplots(figsize = (16,8))
    ax.plot(Y_true, color = gris_uat, label = 'Reales')
    ax.plot(Y_pred, color = azul_uat, label = 'Predicciones')
    
    if(vline == True):
        # only one line may be specified; full height
        plt.axvline(x = len(Y_true) - 25, color = naranja_uat, ls = '--')
    
    ax.set_title(title);
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.ylabel('Velocidad del viento m/s')
    plt.xlabel('Horas')
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    
    #plt.legend(loc='upper right')

In [10]:
def previous_date(current_date, frmt='%Y-%m-%d-%H', days = 1, string=True):
    yesterday = current_date - timedelta(days)
    if string:
        return yesterday.strftime(frmt), current_date.strftime(frmt)
    return yesterday

In [11]:
def get_dates(data, month, day, d = 1):
    current_date = datetime(2022, month, day)
    
    prev, current = previous_date(current_date, days = d + 1, string = True)
    month = data[(data['Date'] > prev) & (data['Date'] < current)]
    
    previous_days = month.iloc[:-24]
    current_day = month.iloc[len(previous_days):]
    
    return month, previous_days, current_day

In [12]:
def simulation(comb, Month, Day, Model, minmax_scl):
    h = int(comb[1] / 24)
    month, p, c = get_dates(df_Prueba, Month, Day, d = h)

    #Realizar una predicciones con metodo recursivo
    #model = LinearRegression()
    
    drop_cols = [col for col in p.columns if col not in comb[0]]
    p.drop(drop_cols, axis = 1, inplace = True)
    
    #model.fit(x_train, y_train)

    win = p
    
    
    if comb[2] > 1:
        # MULTIPLE OUTPUT
        win_scl = minmax_scl.transform(win)
        win_scl = win_scl.reshape(1, -1)
        pred = Model.predict(win_scl)
    else:
        # SINGLE OUTPUT
        win_scl = minmax_scl.transform(win)
        x, pred = recursive_forecast(Model, Ventana = win_scl, Iters = 24, Rolling = False)
        #print(pred)

    if len(comb[0]) > 1:
        #### PARA MULTIPLE INPUT
        # Se instancia un nuevo objeto scaler
        scl_inv = MinMaxScaler()
        # Se obtiene los atributos del scaler utilizado para el fit del modelo
        scl_inv.min_, scl_inv.scale_ = minmax_scl.min_[3], minmax_scl.scale_[3]
        
        # se realiza la transformacion inversa del scaler para los datos de las predicciones
        if comb[2] > 1:
            p_inv = scl_inv.inverse_transform(pred)
        else:
            p_inv = scl_inv.inverse_transform([pred])
        # Se ¿aplana? el arreglo
        p_inv = p_inv.flatten()
    else:
        ### PARA SINGLE INPUT
        # se realiza la transformacion inversa del scaler para los datos de las predicciones
        if comb[2] > 1:
            p_inv = minmax_scl.inverse_transform(pred)
        else:
            p_inv = minmax_scl.inverse_transform([pred])
    p_inv = p_inv.flatten()

    window = p['High Wind Speed m/s'].values
    
    return month, window, c, p_inv

## MSVR

In [13]:
import numpy as np
from numpy import concatenate

# from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
# import statistics

def create_dataset(ts, dim ,h):
    look_back = dim + h -1
    # dataset = np.insert(dataset, [0] * look_back, 0)
    dataX, dataY = [], []
    for i in range(len(ts) - look_back):
        a = ts[i:(i + look_back)]
        dataX.append(a)
        dataY.append(ts[i + look_back])
    dataY = np.array(dataY)
    dataY = np.reshape(dataY, (dataY.shape[0], 1))
    dataset = np.concatenate((dataX, dataY), axis=1)
    return dataset


def unpadding(y):
    a = y.copy()
    h = y.shape[1]
    s = np.empty(y.shape[0] + y.shape[1] -1)

    for i in range(s.shape[0]):
        s[i]=np.diagonal(np.flip(a,1), offset= -i + h-1,axis1=0,axis2=1).copy().mean()
    
    return s

def mape(y_true, y_pred): 
    y_true = unpadding(y_true)
    y_pred = unpadding(y_pred)

    mask =  y_true != 0.0
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)
    N_metric =  (y_true[mask] - y_pred[mask])/y_true[mask]
    N_metric = np.fabs(N_metric)
    metric = N_metric.mean()

    return metric

def smape(y_true, y_pred): 
    y_true = unpadding(y_true)
    y_pred = unpadding(y_pred)

    mask =  y_true != 0.0
    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)
    N_metric =  (y_true[mask] - y_pred[mask])/(y_true[mask] + y_pred[mask])
    N_metric = np.fabs(N_metric)
    metric = N_metric.mean()

def rmse(y_true, y_pred):
    y_true = unpadding(y_true)
    y_pred = unpadding(y_pred)

    return mean_squared_error(y_true,y_pred, squared = False) #np.sqrt(mean_squared_error(y_true,y_pred))

def r2(y_true, y_pred):
    y_true = unpadding(y_true)
    y_pred = unpadding(y_pred)
    
    return r2_score(y_true, y_pred)

def mae(y_true, y_pred):
    y_true = unpadding(y_true)
    y_pred = unpadding(y_pred)
    
    return mean_absolute_error(y_true, y_pred)

In [14]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels

class MSVR():
    def __init__(self, kernel='rbf', degree=3, gamma=None, coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, params = None):
        super(MSVR, self).__init__()
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.tol = tol
        self.C = C
        self.epsilon = epsilon
        self.Beta = None
        self.NSV = None
        self.xTrain = None
        
        if params != None:
            self.C = params['C'][0]
            self.epsilon = params['epsilon'][0]
            self.gamma = params['gamma'][0]
            self.kernel = params['kernel'][0]
        #print(str(self.C))

    def fit(self, x, y):
        self.xTrain = x.copy()
        C = self.C
        epsi = self.epsilon
        tol = self.tol

        n_m = np.shape(x)[0]  # num of samples
        n_d = np.shape(x)[1]  # input data dimensionality
        n_k = np.shape(y)[1]  # output data dimensionality (output variables)

        # H = kernelmatrix(ker, x, x, par)
        H = pairwise_kernels(x, x, metric=self.kernel, filter_params=True,
                             degree=self.degree, gamma=self.gamma, coef0=self.coef0)

        self.Beta = np.zeros((n_m, n_k))

        #E = prediction error per output (n_m * n_k)
        E = y - np.dot(H, self.Beta)
        #RSE
        u = np.sqrt(np.sum(E**2, 1, keepdims=True))

        #RMSE
        RMSE = []
        RMSE_0 = np.sqrt(np.mean(u**2))
        RMSE.append(RMSE_0)

        #points for which prediction error is larger than epsilon
        i1 = np.where(u > epsi)[0]

        #set initial values of alphas a (n_m * 1)
        a = 2 * C * (u - epsi) / u

        #L (n_m * 1)
        L = np.zeros(u.shape)

        # we modify only entries for which  u > epsi. with the sq slack
        L[i1] = u[i1]**2 - 2 * epsi * u[i1] + epsi**2

        #Lp is the quantity to minimize (sq norm of parameters + slacks)
        Lp = []
        BetaH = np.dot(np.dot(self.Beta.T, H), self.Beta)
        Lp_0 = np.sum(np.diag(BetaH), 0) / 2 + C * np.sum(L)/2
        Lp.append(Lp_0)

        eta = 1
        k = 1
        hacer = 1
        val = 1

        while(hacer):
            Beta_a = self.Beta.copy()
            E_a = E.copy()
            u_a = u.copy()
            i1_a = i1.copy()

            M1 = H[i1][:, i1] + \
                np.diagflat(1/a[i1]) + 1e-10 * np.eye(len(a[i1]))

            #compute betas
            #       sal1 = np.dot(np.linalg.pinv(M1),y[i1])  #求逆or广义逆（M-P逆）无法保证M1一定是可逆的？
            sal1 = np.dot(np.linalg.inv(M1), y[i1])

            eta = 1
            self.Beta = np.zeros(self.Beta.shape)
            self.Beta[i1] = sal1.copy()

            #error
            E = y - np.dot(H, self.Beta)
            #RSE
            u = np.sqrt(np.sum(E**2, 1)).reshape(n_m, 1)
            i1 = np.where(u >= epsi)[0]

            L = np.zeros(u.shape)
            L[i1] = u[i1]**2 - 2 * epsi * u[i1] + epsi**2

            #%recompute the loss function
            BetaH = np.dot(np.dot(self.Beta.T, H), self.Beta)
            Lp_k = np.sum(np.diag(BetaH), 0) / 2 + C * np.sum(L)/2
            Lp.append(Lp_k)

            #Loop where we keep alphas and modify betas
            while(Lp[k] > Lp[k-1]):
                eta = eta/10
                i1 = i1_a.copy()

                self.Beta = np.zeros(self.Beta.shape)
                #%the new betas are a combination of the current (sal1)
                #and of the previous iteration (Beta_a)
                self.Beta[i1] = eta*sal1 + (1-eta)*Beta_a[i1]

                E = y - np.dot(H, self.Beta)
                u = np.sqrt(np.sum(E**2, 1)).reshape(n_m, 1)

                i1 = np.where(u >= epsi)[0]

                L = np.zeros(u.shape)
                L[i1] = u[i1]**2 - 2 * epsi * u[i1] + epsi**2
                BetaH = np.dot(np.dot(self.Beta.T, H), self.Beta)
                Lp_k = np.sum(np.diag(BetaH), 0) / 2 + C * np.sum(L)/2
                Lp[k] = Lp_k

                #stopping criterion 1
                if(eta < 1e-16):
                    Lp[k] = Lp[k-1] - 1e-15
                    self.Beta = Beta_a.copy()

                    u = u_a.copy()
                    i1 = i1_a.copy()

                    hacer = 0

            #here we modify the alphas and keep betas
            a_a = a.copy()
            a = 2 * C * (u - epsi) / u

            RMSE_k = np.sqrt(np.mean(u**2))
            RMSE.append(RMSE_k)

            if((Lp[k-1]-Lp[k])/Lp[k-1] < tol):
                hacer = 0

            k = k + 1

            #stopping criterion #algorithm does not converge. (val = -1)
            if(len(i1) == 0):
                hacer = 0
                self.Beta = np.zeros(self.Beta.shape)
                val = -1

        self.NSV = len(i1)

    def predict(self, x):
        H = pairwise_kernels(x, self.xTrain, metric=self.kernel, filter_params=True,
                             degree=self.degree, gamma=self.gamma, coef0=self.coef0)
        yPred = np.dot(H, self.Beta)
        return yPred

    # def score(self,x):

In [15]:
def scores_cv(model, x_, y_, cv, verbose = False):
    cv_results = cross_validate(
        model,
        x_,
        y_,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error", "r2"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    r2 = cv_results["test_r2"]
    if verbose == True:
        print(
            f"Coefficient of determination: {r2.mean():.3f} +/- {r2.std():.3f}\n"
            f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}\n"
            f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        )
    return {'r2': r2.mean(), 'rmse': rmse.mean(), 'mae': mae.mean()}

In [16]:
# Lista de combinaciones para los experimentos
list1 = [['Month', 'Day', 'Hour', 'High Wind Speed m/s'], ['High Wind Speed m/s']]
list2 = [24, 48, 72, 96]
list3 = [1, 24]
listcomb = [list1, list2, list3]
# Realiza las distitnas combinaciones
combinations = [p for p in itertools.product(*listcomb)]
# Elimina las combinaciones SIMO
for idx, comb in enumerate(combinations):
    if comb[2] == 24 and comb[0] == 'High Wind Speed m/s':
        combinations.pop(idx)

# Agrega cada combinacion a su respectiva lista
list_rect = []
list_mimo = []
for comb in combinations:
    if comb[2] == 1:
        list_rect.append(comb)
    else:
        list_mimo.append(comb)

In [17]:
def CV(x_, y_, tscv, params, key):
    best_param = 0
    best_score = -9999
    testmetric = []
    #print(key)
    #print(params[])
    for param in params[key]:
        #print(param)
        mean_r2 = 0.0
        params.update({key: [param]})
        for k, (train_index, test_index) in enumerate(tscv.split(x_)):
            #print(f"Fold {k}:")
            #print(f"  Train: index={train_index}")
            #print(f"  Test:  index={test_index}")
            #print(params)
            msvr = MSVR(params = params)
            msvr.fit(x_[train_index], y_[train_index])

            testPred = msvr.predict(x_[test_index])
            testr2 = r2(y_[test_index], testPred)
            testmetric.append(testr2)

        sum_r2 = sum(testmetric)
        mean_r2 = sum_r2 / len(testmetric)
        #mean_r2 += 1
        if best_score < mean_r2:
            best_score = mean_r2
            best_param = param
    print(str(best_param) + " : " + str(best_score))
    return best_param

In [19]:
%%time
### USAR PARA METODO MIMO
### REALIZAR EL GS DE MANERA MANUAL, PARA CADA PARAMETRO REALIZAR UN CICLO HASTA COMPLETAR LAS 3 ETAPAS Y LOS 8 EXPERIMENTOS
### CON VALIDACION CRUZADA
# Parametros del modelo

#####ESTA CONVIRTIENDO EL BESTPARAM A STR {{{ARREGLAR}}}#####
params = {'C': [1, 10, 100, 1000], 'epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 1.5, 2], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 1.5, 2],'kernel': ['rbf']}
keys = ['epsilon', 'C', 'gamma']

#model = MSVR()
tscv = TimeSeriesSplit(n_splits = 4)
minmax_scl = MinMaxScaler()

for i, comb in enumerate(list_mimo):
    df_train = df_Nizanda.copy()
    drop_cols = [col for col in df_Nizanda.columns if col not in comb[0]]
    df_train.drop(drop_cols, axis = 1, inplace = True)
    minmax_scl.fit(df_train)

    window = comb[1]
    horizon = comb[2]
    shift = 0
    w, n_cols = data_preprocessing(df_train, minmax_scl)
    x_, y_ = make_windows(w, input_size = window, horizon = horizon, shift = shift, n_cols = n_cols)
    
    if len(x_.shape) > 2:
        # Convertir a arreglo de 2 dimensiones
        rows, nx, ny = x_.shape
        x_ = x_.reshape((rows, nx*ny))
    
    print('---------------Inicio de experimento: '+ str(i) +' -----------------')
    
    bestparams = {'C': [1], 'epsilon': [0.1], 'gamma': [0.1],'kernel': ['rbf']}
    for j in range(3):
        print('--Etapa ' + str(j + 1) + '--')
        print('GS: '+ str(keys[j]))
        bestparams.update({keys[j]: params[keys[j]]})
        print(bestparams)
        best_param = CV(x_, y_, tscv, bestparams, keys[j])
        #print(best_param)
        bestparams.update({keys[j]: [best_param]})
        print('Parametro acutalizado: ' + str(bestparams))
    
    print('Parametros del experimento ' + str(i + 1) + ': ' + str(bestparams))
    
    # CROSS VALIDATION
    tr2 = []
    trmse = []
    tmae = []
    score_r2 = 0
    score_rmse = 0
    score_mae = 0

    for k, (train_index, test_index) in enumerate(tscv.split(x_)):
        msvr = MSVR(params = bestparams)
        msvr.fit(x_[train_index], y_[train_index])

        testPred = msvr.predict(x_[test_index])

        testr2 = r2(y_[test_index], testPred)
        tr2.append(testr2)

        testrmse = rmse(y_[test_index], testPred)
        trmse.append(testrmse)

        testmae = mae(y_[test_index], testPred)
        tmae.append(testmae)

    sum_r2 = sum(tr2)
    score_r2 = sum_r2 / len(tr2)

    sum_rmse = sum(trmse)
    score_rmse = sum_rmse / len(trmse)

    sum_mae = sum(tmae)
    score_mae = sum_mae / len(tmae)
    
    
    #metrics = scores_cv(SVR(C = bestparams['C'][0], epsilon = bestparams['epsilon'][0], gamma = bestparams['gamma'][0]), x_, y_, tscv, verbose = False)
    df_SVR_mimo_test.loc[len(df_SVR_mimo_test.index)] = ['MSVR', comb[2], str(score_r2), str(score_rmse), str(score_mae), [comb[0], comb[1], comb[2]], bestparams]
    
    print('---------------Fin de experimento: '+ str(i) +' -----------------')
print('---------------Lote de experimentos terminados -----------------')

---------------Inicio de experimento: 0 -----------------
--Etapa 1--
GS: epsilon
{'C': [1], 'epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 1.5, 2], 'gamma': [0.1], 'kernel': ['rbf']}
0.1 : 0.39332463728234995
Parametro acutalizado: {'C': [1], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
--Etapa 2--
GS: C
{'C': [1, 10, 100, 1000], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
1 : 0.4020062801264353
Parametro acutalizado: {'C': [1], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
--Etapa 3--
GS: gamma
{'C': [1], 'epsilon': [0.1], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 1.5, 2], 'kernel': ['rbf']}
0.1 : 0.37376676797982494
Parametro acutalizado: {'C': [1], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
Parametros del experimento 1: {'C': [1], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}


C:\Users\Angel\anaconda3\envs\Tesis_Env\lib\site-packages\pandas\core\dtypes\cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


---------------Fin de experimento: 0 -----------------
---------------Inicio de experimento: 1 -----------------
--Etapa 1--
GS: epsilon
{'C': [1], 'epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 1.5, 2], 'gamma': [0.1], 'kernel': ['rbf']}
0.1 : 0.07791870782869185
Parametro acutalizado: {'C': [1], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
--Etapa 2--
GS: C
{'C': [1, 10, 100, 1000], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
1 : 0.09459370407098233
Parametro acutalizado: {'C': [1], 'epsilon': [0.1], 'gamma': [0.1], 'kernel': ['rbf']}
--Etapa 3--
GS: gamma
{'C': [1], 'epsilon': [0.1], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 1.5, 2], 'kernel': ['rbf']}
0.01 : 0.3506414252808035
Parametro acutalizado: {'C': [1], 'epsilon': [0.1], 'gamma': [0.01], 'kernel': ['rbf']}
Parametros del experimento 2: {'C': [1], 'epsilon': [0.1], 'gamma': [0.01], 'kernel': ['rbf']}
---------------Fin de experimento: 1 -----------------
---------------Inicio de experimento: 2 -----------------
--Etap

In [20]:
df_SVR_mimo_test

,Modelo,Horizon,R2,RMSE,MAE,Comb,Params
0,MSVR,24,0.4020062801264353,0.13123946528419278,0.10233649277894294,"[[Month, Day, Hour, High Wind Speed m/s], 24, 24]","{'C': [1], 'epsilon': [0.1], 'gamma': [0.1], '..."
1,MSVR,24,0.4266997400264012,0.128360359299714,0.09942853181297097,"[[Month, Day, Hour, High Wind Speed m/s], 48, 24]","{'C': [1], 'epsilon': [0.1], 'gamma': [0.01], ..."
2,MSVR,24,0.40423452981735575,0.13123933053902578,0.1016586262470574,"[[Month, Day, Hour, High Wind Speed m/s], 72, 24]","{'C': [1], 'epsilon': [0.1], 'gamma': [0.01], ..."
3,MSVR,24,0.3563883694544089,0.13610664121795266,0.10564005190717199,"[[Month, Day, Hour, High Wind Speed m/s], 96, 24]","{'C': [1], 'epsilon': [0.01], 'gamma': [0.01],..."
4,MSVR,24,0.5928641995782982,0.11015661905086542,0.08452892130215875,"[[High Wind Speed m/s], 24, 24]","{'C': [100], 'epsilon': [0.0001], 'gamma': [0...."
5,MSVR,24,0.5785733090299188,0.11214125069320324,0.0855592193873844,"[[High Wind Speed m/s], 48, 24]","{'C': [10], 'epsilon': [0.0001], 'gamma': [0.1..."
6,MSVR,24,0.5617799439801998,0.11472870660575972,0.08798977043367864,"[[High Wind Speed m/s], 72, 24]","{'C': [1], 'epsilon': [0.1], 'gamma': [0.1], '..."
7,MSVR,24,0.5451967234789215,0.11739883166272032,0.08996098080891721,"[[High Wind Speed m/s], 96, 24]","{'C': [1], 'epsilon': [0.1], 'gamma': [0.1], '..."


In [27]:
df_SVR_mimo_test.to_csv(r'C:\Users\Angel\Documents\Tesis\Notebooks\Resultados\SVR_MIMO_V2.csv')